## TechPoint Xtern WSA (Part 2)

In [1]:
import googlemaps
from googlemaps import directions
from googlemaps import maps
import pandas as pd
from IPython.display import Image

gmaps = googlemaps.Client(key=open('apikey.txt', 'r').read())

API functions:
- https://github.com/googlemaps/google-maps-services-python/blob/master/googlemaps/places.py
- https://github.com/googlemaps/google-maps-services-python/blob/master/googlemaps/directions.py
- https://github.com/googlemaps/google-maps-services-python/blob/master/googlemaps/maps.py

### Retrieving candidates
I saved the output file to a .xlsx file, selected destinations based on my tastes using the **select** column, and imported it below.

In [2]:
cand = pd.read_excel('ancillary/cand.xlsx')

cand = cand.loc[cand.select.notna()]
cand

,select,name,address,rating,rating_count,type,lat,lng,price_level,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,website
2,y,Harry & Izzy's,"153 S Illinois St, Indianapolis, IN 46225, Uni...",4.7,2962,restaurant,39.764575,-86.159660,4.0,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 11:00 PM,11:00 AM - 11:00 PM,12:00 - 9:00 PM,http://www.harryandizzys.com/
26,y,Sub Zero Nitrogen Ice Cream,"427 Massachusetts Ave, Indianapolis, IN 46204,...",4.4,1182,ice cream,39.773126,-86.151333,2.0,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 10:00 PM,12:00 - 10:00 PM,12:00 - 8:00 PM,https://www.subzeroicecream.com/
28,y,Silver In the City,"434 Massachusetts Ave, Indianapolis, IN 46204,...",4.7,414,gift shop,39.773386,-86.151706,NaN,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 9:00 PM,10:00 AM - 9:00 PM,10:00 AM - 6:00 PM,http://www.silverinthecity.com/
52,y,Indiana War Memorial & Museum,"55 E Michigan St, Indianapolis, IN 46204, Unit...",4.7,2372,museum,39.773361,-86.156951,NaN,Closed,Closed,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,https://www.indianawarmemorials.org/
55,y,Indiana State Museum,"650 W Washington St, Indianapolis, IN 46204, U...",4.6,3335,museum,39.768392,-86.169501,NaN,Closed,Closed,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,http://www.indianamuseum.org/
56,y,Eiteljorg Museum,"500 W Washington St, Indianapolis, IN 46204, U...",4.6,1657,museum,39.768575,-86.167791,NaN,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,12:00 - 5:00 PM,http://www.eiteljorg.org/
59,y,Soldiers & Sailors Monument,"1 Monument Cir, Indianapolis, IN 46204, United...",4.7,8028,tourist,39.768546,-86.158044,NaN,Closed,Closed,Closed,Closed,10:30 AM - 5:30 PM,10:30 AM - 5:30 PM,10:30 AM - 5:30 PM,http://www.in.gov/iwm
60,y,Indianapolis Canal Walk,"Canal Walk, Indianapolis, IN 46202, United States",4.7,2288,tourist,39.775204,-86.165168,NaN,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,https://www.whiteriverstatepark.org/


### Finding optimal route

Similar to this example: https://www.linkedin.com/pulse/find-optimal-route-using-google-maps-api-andre-cedras/

Please note, when using the directions.directions() function:
- $5 USD per 1000 requests ( 1-10 waypoints )
- $10 USD per 1000 requests ( 11-25 waypoints )
- Maximum allowed number of waypoints per request: 25
- Ref: https://developers.google.com/maps/documentation/directions/usage-and-billing?hl=en_US

In [3]:
cand['coords'] = [(a,b) for a,b in zip(cand.lat, cand.lng)]

origin_name = 'Indianapolis Canal Walk'
dest_name = "Harry & Izzy's"


origin = tuple(cand.loc[cand.name == origin_name,'coords'])[0]
dest = tuple(cand.loc[cand.name == dest_name,'coords'])[0]

wp = []

i = 0
for a in cand['coords']:
    if a != origin and a != dest:
        wp.append(i)
        i += 1
    elif a == origin:
        wp.append(-1)
    else:
        wp.append(cand.shape[0]-2)

cand.index = wp
# cand

In [4]:
route = directions.directions(client = gmaps, origin=origin, destination=dest, waypoints=[a for a in cand['coords'] if a != origin and a != dest], optimize_waypoints=True, mode='walking')
route[0]['waypoint_order']

[3, 4, 2, 1, 0, 5]

In [5]:
df = cand.reindex([-1] + route[0]['waypoint_order'] + [cand.shape[0]-2]).reset_index()
df = df[['name','address','coords','rating','website','type','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]
df

,name,address,coords,rating,website,type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Indianapolis Canal Walk,"Canal Walk, Indianapolis, IN 46202, United States","(39.7752036, -86.165168)",4.7,https://www.whiteriverstatepark.org/,tourist,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM
1,Indiana State Museum,"650 W Washington St, Indianapolis, IN 46204, U...","(39.7683918999999, -86.1695013)",4.6,http://www.indianamuseum.org/,museum,Closed,Closed,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM
2,Eiteljorg Museum,"500 W Washington St, Indianapolis, IN 46204, U...","(39.7685753999999, -86.1677914999999)",4.6,http://www.eiteljorg.org/,museum,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,12:00 - 5:00 PM
3,Indiana War Memorial & Museum,"55 E Michigan St, Indianapolis, IN 46204, Unit...","(39.7733607, -86.1569507999999)",4.7,https://www.indianawarmemorials.org/,museum,Closed,Closed,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM
4,Silver In the City,"434 Massachusetts Ave, Indianapolis, IN 46204,...","(39.7733855, -86.1517061999999)",4.7,http://www.silverinthecity.com/,gift shop,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 9:00 PM,10:00 AM - 9:00 PM,10:00 AM - 6:00 PM
5,Sub Zero Nitrogen Ice Cream,"427 Massachusetts Ave, Indianapolis, IN 46204,...","(39.7731265, -86.151333)",4.4,https://www.subzeroicecream.com/,ice cream,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 10:00 PM,12:00 - 10:00 PM,12:00 - 8:00 PM
6,Soldiers & Sailors Monument,"1 Monument Cir, Indianapolis, IN 46204, United...","(39.7685459, -86.1580437)",4.7,http://www.in.gov/iwm,tourist,Closed,Closed,Closed,Closed,10:30 AM - 5:30 PM,10:30 AM - 5:30 PM,10:30 AM - 5:30 PM
7,Harry & Izzy's,"153 S Illinois St, Indianapolis, IN 46225, Uni...","(39.764575, -86.15966)",4.7,http://www.harryandizzys.com/,restaurant,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 11:00 PM,11:00 AM - 11:00 PM,12:00 - 9:00 PM


Some final manual adjustments to the route, to account for closing times:

In [6]:
def move_row_up(data, old, new):
    temp = pd.DataFrame(data.loc[old]).T
    data = data.drop(old)
    if old <= new:
        data = pd.concat([data.loc[:new], temp, data.loc[new+1:]]).reset_index()
    else:
        data = pd.concat([data.loc[:new-1], temp, data.loc[new:]]).reset_index()
    del data['index']
    return data

In [7]:
df = move_row_up(df,4,6)
df = move_row_up(df,4,5)
df

,name,address,coords,rating,website,type,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Indianapolis Canal Walk,"Canal Walk, Indianapolis, IN 46202, United States","(39.7752036, -86.165168)",4.7,https://www.whiteriverstatepark.org/,tourist,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM,7:00 AM - 8:00 PM
1,Indiana State Museum,"650 W Washington St, Indianapolis, IN 46204, U...","(39.7683918999999, -86.1695013)",4.6,http://www.indianamuseum.org/,museum,Closed,Closed,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM
2,Eiteljorg Museum,"500 W Washington St, Indianapolis, IN 46204, U...","(39.7685753999999, -86.1677914999999)",4.6,http://www.eiteljorg.org/,museum,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,10:00 AM - 5:00 PM,12:00 - 5:00 PM
3,Indiana War Memorial & Museum,"55 E Michigan St, Indianapolis, IN 46204, Unit...","(39.7733607, -86.1569507999999)",4.7,https://www.indianawarmemorials.org/,museum,Closed,Closed,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM,9:00 AM - 5:00 PM
4,Soldiers & Sailors Monument,"1 Monument Cir, Indianapolis, IN 46204, United...","(39.7685459, -86.1580437)",4.7,http://www.in.gov/iwm,tourist,Closed,Closed,Closed,Closed,10:30 AM - 5:30 PM,10:30 AM - 5:30 PM,10:30 AM - 5:30 PM
5,Sub Zero Nitrogen Ice Cream,"427 Massachusetts Ave, Indianapolis, IN 46204,...","(39.7731265, -86.151333)",4.4,https://www.subzeroicecream.com/,ice cream,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 9:00 PM,1:00 - 10:00 PM,12:00 - 10:00 PM,12:00 - 8:00 PM
6,Silver In the City,"434 Massachusetts Ave, Indianapolis, IN 46204,...","(39.7733855, -86.1517061999999)",4.7,http://www.silverinthecity.com/,gift shop,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 8:00 PM,10:00 AM - 9:00 PM,10:00 AM - 9:00 PM,10:00 AM - 6:00 PM
7,Harry & Izzy's,"153 S Illinois St, Indianapolis, IN 46225, Uni...","(39.764575, -86.15966)",4.7,http://www.harryandizzys.com/,restaurant,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 10:00 PM,11:00 AM - 11:00 PM,11:00 AM - 11:00 PM,12:00 - 9:00 PM


In [8]:
route = directions.directions(client = gmaps, origin=origin, destination=dest, waypoints=[a for a in df['coords'] if a != origin and a != dest], optimize_waypoints=False, mode='walking')

del df['coords']
df.to_csv('ancillary/plan_raw.csv')
# df

### Mapping the route

Using method found in: https://www.codeforests.com/2021/01/23/plot-route-on-google-maps/

Please note, when using maps.static_map() function:
- $2 USD per 1000 requests
- Ref: https://developers.google.com/maps/documentation/maps-static/usage-and-billing?hl=en_US

In [9]:
marker_points = []
waypoints = []

for leg in route[0]['legs']:
    leg_start_loc = leg['start_location']
    marker_points.append(f"{leg_start_loc['lat']},{leg_start_loc['lng']}")
    for step in leg['steps']:
        end_loc = step['end_location']
        waypoints.append(f"{end_loc['lat']},{end_loc['lng']}")
last_stop = route[0]['legs'][-1]['end_location']
marker_points.append(f"{last_stop['lat']},{last_stop['lng']}")
        
markers = [ 'color:blue|size:mid|label:' + str(i) + '|' + r for i, r in enumerate(marker_points)]
result_map = maps.static_map(client = gmaps,
                 center = waypoints[0],
                 scale=4, 
                 zoom=14,
                 size=[640, 640], 
                 format='png32', 
                 maptype='roadmap',
                 markers=markers,
                 path='color:0x0000ff|weight:2|' + '|'.join(waypoints))
with open('img/cand_route.png', 'wb') as img:
    for chunk in result_map:
        img.write(chunk)

In [2]:
Image(url="img/cand_route.png", width=800, height=800)

All that's left for me to do (in Excel) is filling in the details of the itinerary: starting times, durations, and activity types.

### Conclusions

Creating an itinerary is highly subjective and difficult to automate. Fortunately, I believe I've demonstrated how we can simplify the task.

The first step is systematically narrow down destinations by analyzing user ratings, so that we are not overwhelmed by the sheer number of choices.

The second step is to find an optimized route to simplify scheduling and minimize time spent between destinations (cannot be more than 25 waypoints).

This process allows for lots of customization based on personal tastes, so anyone can use this code to narrow down destinations and create a schedule with an optimized route.